In [1]:
import os
import torch
import pickle
import torch.nn.functional as F
from torch.utils.data import DataLoader
from transformers import AutoTokenizer, AutoModel, AutoModelForMaskedLM
import numpy as np
from sklearn.manifold import TSNE
import matplotlib.pyplot as plt
import seaborn as sns
from scipy.spatial.distance import euclidean

def get_embedding(text, tokenizer, model, device):
    inputs = tokenizer(text, return_tensors='pt')
    inputs = {k: v.to(device) for k, v in inputs.items()}
    with torch.no_grad():
        outputs = model(**inputs, output_hidden_states=True)
    hidden_states = outputs.hidden_states[-1]  
    return hidden_states.mean(dim=1).cpu().numpy().flatten()
    
input_pickle_path = 'DataSet_triplet/PUNJAB_05_without_city.pkl'

with open(input_pickle_path, 'rb') as f:
    data = pickle.load(f)

triplets = data.get('triplets', [])
addresses = data.get('addresses', [])
lat_lngs = data.get('lat_lngs', [])

print(f"Length of triplets: {len(triplets)}, {triplets[-1]}")
print(f"Length of addresses: {len(addresses)}, {addresses[-1]}")
print(f"Length of lat_lngs: {len(lat_lngs)}, {lat_lngs[-1]}")

def load_finetuned_model(checkpoint_path, base_tokenizer_path):
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    model = AutoModelForMaskedLM.from_pretrained(checkpoint_path, use_safetensors=True)
    tokenizer = AutoTokenizer.from_pretrained(base_tokenizer_path)
    model = model.to(device)
    return model, tokenizer, device

def get_embeddings(texts, tokenizer, model, device):
    embeddings = [get_embedding(text, tokenizer, model, device) for text in texts]
    return np.vstack(embeddings)

def print_euclidean_distances(addresses, triplets, tokenizer, model, device, num_triplets=10):
    subset_triplets = triplets[:num_triplets]
    for i, (a, p, n, _) in enumerate(subset_triplets):
        
        anchor_emb = get_embedding(addresses[a], tokenizer, model, device)
        positive_emb = get_embedding(addresses[p], tokenizer, model, device)
        negative_emb = get_embedding(addresses[n], tokenizer, model, device)

        pos_distance = euclidean(anchor_emb, positive_emb)
        neg_distance = euclidean(anchor_emb, negative_emb)

        print(f"Triplet {i+1}:")
        print(subset_triplets[i])
        print(f"  Anchor-Positive Distance: {pos_distance}")
        print(f"  Anchor-Negative Distance: {neg_distance}")
        print(f"  Difference: {neg_distance - pos_distance}\n")


checkpoint_path = "/home/uddeshya.singh/Experiments/results_new/checkpoint-180000"
base_tokenizer_path = "/home/devanapalli.ravi/disk_1/paper/data/gpu_re_trained_models/lm_retrained_models/pan_india_train_threshold/pan_india_train_threshold_out_dir_5_epochs/checkpoint-375000"
model, tokenizer, device = load_finetuned_model(checkpoint_path, base_tokenizer_path)

num_triplets_to_use = 10
print_euclidean_distances(addresses, triplets, tokenizer, model, device, num_triplets=num_triplets_to_use)


/home/uddeshya.singh/miniconda3/envs/uddeshya_env/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Length of triplets: 3217904, (1114168, 1114167, 1114099, 3)
Length of addresses: 1114180, wardno 1 vpo sarhala ranuan
Length of lat_lngs: 1114180, [31.2044125, 75.8583784]
Triplet 1:
(0, 533, 574, 3)
  Anchor-Positive Distance: 13.32730484008789
  Anchor-Negative Distance: 17.5146427154541
  Difference: 4.187337875366211

Triplet 2:
(59, 533, 527, 3)
  Anchor-Positive Distance: 20.320056915283203
  Anchor-Negative Distance: 21.53264617919922
  Difference: 1.2125892639160156

Triplet 3:
(533, 0, 527, 3)
  Anchor-Positive Distance: 13.32730484008789
  Anchor-Negative Distance: 11.578231811523438
  Difference: -1.7490730285644531

Triplet 4:
(1, 290, 116, 1)
  Anchor-Positive Distance: 12.3921537399292
  Anchor-Negative Distance: 15.570472717285156
  Difference: 3.178318977355957

Triplet 5:
(1, 290, 420, 2)
  Anchor-Positive Distance: 12.3921537399292
  Anchor-Negative Distance: 12.550565719604492
  Difference: 0.15841197967529297

Triplet 6:
(1, 290, 29, 3)
  Anchor-Positive Distance: 1

In [2]:
from transformers import AutoModelForMaskedLM, AutoTokenizer

model_save_path = "/home/uddeshya.singh/Experiments/Model_conversion/Model_1_R_without_city"
model.save_pretrained(model_save_path)
tokenizer.save_pretrained(model_save_path)

('/home/uddeshya.singh/Experiments/Model_conversion/Model_1_R_without_city/tokenizer_config.json',
 '/home/uddeshya.singh/Experiments/Model_conversion/Model_1_R_without_city/special_tokens_map.json',
 '/home/uddeshya.singh/Experiments/Model_conversion/Model_1_R_without_city/vocab.json',
 '/home/uddeshya.singh/Experiments/Model_conversion/Model_1_R_without_city/merges.txt',
 '/home/uddeshya.singh/Experiments/Model_conversion/Model_1_R_without_city/added_tokens.json',
 '/home/uddeshya.singh/Experiments/Model_conversion/Model_1_R_without_city/tokenizer.json')

In [ ]:
import torch
from transformers import AutoModel, AutoTokenizer

def load_finetuned_model(checkpoint_path):
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    model = AutoModel.from_pretrained(checkpoint_path)
    tokenizer = AutoTokenizer.from_pretrained(checkpoint_path)
    model = model.to(device)
    return model, tokenizer, device